In [3]:
import agate

In [4]:
crashes2015 = agate.Table.from_csv('crashes2015.csv')

In [5]:

persons2015 = agate.Table.from_csv('persons2015.csv')

In [6]:

vehicles2015 = agate.Table.from_csv('vehicles2015.csv')

In [7]:
joined_crash_person=crashes2015.join(persons2015, 'CrashId', 'CrashId', inner=True)

In [8]:
joined_all = joined_crash_person.join(vehicles2015, 'CrashId', 'CrashId', inner = True)

In [9]:
print joined_all

|------------------------------+---------------|
|  column_names                | column_types  |
|------------------------------+---------------|
|  CrashId                     | Number        |
|  AgencyOri                   | Text          |
|  AgencyCaseNumber            | Number        |
|  CrashType                   | Number        |
|  CrashDate                   | DateTime      |
|  CrashDateUnk                | Text          |
|  InvestigationDate           | DateTime      |
|  ResponseMinuites            | Number        |
|  ResponseTimeUnk             | Text          |
|  NumberInjured               | Number        |
|  NumberMinorInjuries         | Number        |
|  NumberDisablingInjuries     | Number        |
|  NumberKilled                | Number        |
|  County                      | Number        |
|  Municipality                | Number        |
|  BeatZone                    | Text          |
|  TroopDistrict               | Text          |
|  OnOffRoadway     

In [10]:
joined_all.print_table().limit(10)

|--------------+-----------+------------------+-----------+---------------------+--------------+------|
|      CrashId | AgencyOri | AgencyCaseNumber | CrashType |           CrashDate | CrashDateUnk | ...  |
|--------------+-----------+------------------+-----------+---------------------+--------------+------|
|  150,010,283 | MO0100000 |       20,151,811 |         3 | 2015-04-17 23:51:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015

AttributeError: 'NoneType' object has no attribute 'limit'

In [11]:
joined_all_10Lines=joined_all.limit(10)

In [12]:
joined_all_10Lines.print_table()

|--------------+-----------+------------------+-----------+---------------------+--------------+------|
|      CrashId | AgencyOri | AgencyCaseNumber | CrashType |           CrashDate | CrashDateUnk | ...  |
|--------------+-----------+------------------+-----------+---------------------+--------------+------|
|  150,010,283 | MO0100000 |       20,151,811 |         3 | 2015-04-17 23:51:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015-01-26 21:20:00 |              | ...  |
|  150,014,584 | MO0100400 |  201,500,003,032 |         9 | 2015

In [13]:
Distinct_CrashId = joined_all.distinct('CrashId')

In [14]:
num_crashid=Distinct_CrashId.aggregate(agate.Count('CrashId'))

In [15]:
print num_crashid

1380


In [16]:
## number of crashes was 1380 in 2015 in Columbia

In [17]:
## get fatal crashes from joined_all
fatal=joined_all.where(lambda row: row['PersonalInjuryLevel'] =="1")

In [18]:
## get number of fatal crashes
num_crashid=fatal.aggregate(agate.Count('CrashId'))

In [19]:
print num_crashid

11


In [20]:
##There were 11 fatal crashes in 2015. (Correction will come later)

In [21]:
## get number of fatalities
distinct_fatalities = fatal.distinct('PersonId')

In [22]:
Num_fatalities=distinct_fatalities.aggregate(agate.Count('PersonId'))
print Num_fatalities

8


In [23]:
## There were 8 fatalities in 2015.

In [24]:
## get number of fatal crashes adding distinct
fatal_distinct_crashid=fatal.distinct('CrashId')
num_fatal_crash=fatal_distinct_crashid.aggregate(agate.Count('CrashId'))
print num_fatal_crash

6


In [25]:
## Okay the number of fatal crashes was actually 6, the 11 included duplicates

In [26]:
## group by CrashType in distinct fatal crashids
by_crashtype = fatal_distinct_crashid.group_by('CrashType')


In [27]:
crashtype_totals = by_crashtype.aggregate([
    ('count', agate.Count())
])

sorted_crashtype_totals = crashtype_totals.order_by('count', reverse=True)

sorted_crashtype_totals.print_table()

|------------+--------|
|  CrashType | count  |
|------------+--------|
|          3 |     3  |
|          7 |     2  |
|          5 |     1  |
|------------+--------|


In [28]:
## In the 6 fatal crashes, 3 cases involved fixed objects, 2 motor vehicle, 1 pedestrian.

In [29]:
## Let's look at the CrashType of all crashes (of all severities)
by_crashtype = Distinct_CrashId.group_by('CrashType')
crashtype_totals = by_crashtype.aggregate([
    ('count', agate.Count())
])

sorted_crashtype_totals = crashtype_totals.order_by('count', reverse=True)

sorted_crashtype_totals.print_table()

|------------+--------|
|  CrashType | count  |
|------------+--------|
|          7 | 1,023  |
|          3 |   216  |
|          9 |    53  |
|          5 |    22  |
|          2 |    22  |
|         10 |    17  |
|          1 |     9  |
|          4 |     5  |
|         11 |     5  |
|         16 |     3  |
|         13 |     2  |
|         17 |     1  |
|         18 |     1  |
|         15 |     1  |
|------------+--------|


In [30]:
## In all crashes with any kind of severity, 1023 cases involved collision between motor vehicles, 216 fixed objects,  53 parked vehicle

In [31]:
## look at what kind of fixed objects they ran into
## first see if in any case cars hit two objects
with_objectcd2 = joined_all.where(lambda row: row['FixedObjectCd2'] is not None)
with_objectcd2.print_table()

|----------------+-----------+------------------+-----------+---------------------+--------------+------|
|        CrashId | AgencyOri | AgencyCaseNumber | CrashType |           CrashDate | CrashDateUnk | ...  |
|----------------+-----------+------------------+-----------+---------------------+--------------+------|
|    150,037,763 | MO0100400 |  201,500,013,668 |         3 | 2015-05-02 02:15:00 |              | ...  |
|    150,021,217 | MO0100400 |  201,500,003,787 |         3 | 2015-02-01 00:00:00 |              | ...  |
|    150,026,154 | MO0100000 |       20,151,245 |         3 | 2015-03-17 12:00:00 |              | ...  |
|  2,150,007,675 | MO0100200 |    2,015,018,449 |         3 | 2015-01-30 06:50:00 |              | ...  |
|  2,150,002,429 | MO0100200 |    2,015,015,784 |         3 | 2015-01-26 15:08:00 |              | ...  |
|  2,150,002,429 | MO0100200 |    2,015,015,784 |         3 | 2015-01-26 15:08:00 |              | ...  |
|  2,150,006,488 | MO0100200 |    2,015,042,29

In [32]:
crashid150037763 = joined_all.where(lambda row: row['CrashId'] == 150037763)
crashid150037763.print_table()

|--------------+-----------+------------------+-----------+---------------------+--------------+------|
|      CrashId | AgencyOri | AgencyCaseNumber | CrashType |           CrashDate | CrashDateUnk | ...  |
|--------------+-----------+------------------+-----------+---------------------+--------------+------|
|  150,037,763 | MO0100400 |  201,500,013,668 |         3 | 2015-05-02 02:15:00 |              | ...  |
|--------------+-----------+------------------+-----------+---------------------+--------------+------|


In [33]:
column =crashid150037763.columns['FixedObjectCd2']
print column[0]

29


In [35]:
## create a new table with only two columns: CrashId, Object
fixed_objectcd1 = joined_all.select(('CrashId', 'FixedObjectCd1'))
object1=fixed_objectcd1.rename(column_names=['CrashId','FixedObject'])

In [57]:
fixed_objectcd2 = joined_all.select(('CrashId', 'FixedObjectCd2'))
object2=fixed_objectcd2.rename(column_names=['CrashId','FixedObject'])
fixed_objectcd3 = joined_all.select(('CrashId', 'FixedObjectCd3'))
object3=fixed_objectcd3.rename(column_names=['CrashId','FixedObject'])

In [53]:
print(fixed_objectcd3)

|-----------------+---------------|
|  column_names   | column_types  |
|-----------------+---------------|
|  CrashId        | Number        |
|  FixedObjectCd3 | Number        |
|-----------------+---------------|



In [59]:
print object1

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  CrashId      | Number        |
|  FixedObject  | Number        |
|---------------+---------------|



In [75]:
## Went back to the tables and found that FixedObjectcd 4-8 are null, and are recognized as booleans in agate
objects = agate.TableSet([
        object1,
        object2,
        object3,
    ],
        ['cd1', 'cd2', 'cd3'],
    key_name='cd_num'
                        )

FixedObject = objects.merge(groups=None, group_name=None, group_type=None)

In [78]:
FixedObject.print_table()

|---------+-------------+--------------|
|  cd_num |     CrashId | FixedObject  |
|---------+-------------+--------------|
|  cd1    | 150,010,283 |          20  |
|  cd1    | 150,014,584 |              |
|  cd1    | 150,014,584 |              |
|  cd1    | 150,014,584 |              |
|  cd1    | 150,014,584 |              |
|  cd1    | 150,014,584 |              |
|  cd1    | 150,014,584 |              |
|  cd1    | 150,016,788 |              |
|  cd1    | 150,016,788 |              |
|  cd1    | 150,016,788 |              |
|  cd1    | 150,016,788 |              |
|  cd1    | 150,016,788 |              |
|  cd1    | 150,016,788 |              |
|  cd1    | 150,020,791 |              |
|  cd1    | 150,020,791 |              |
|  cd1    | 150,020,791 |              |
|  cd1    | 150,020,791 |              |
|  cd1    | 150,021,080 |              |
|  cd1    | 150,021,080 |              |
|  cd1    | 150,021,080 |              |
|  ...    |         ... |         ...  |
|---------+-----

In [76]:
FixedObject_transformed = objects.pivot('FixedObject')

In [77]:
FixedObject_transformed.print_table()

AttributeError: Table method "print_table" cannot be used as a TableSet method.

In [43]:
## reimport the tables with the right data type
tester = agate.TypeTester(force={
        ['FixedObjectCd1': agate.text(),
        'FixedObjectCd2': agate.text(),
        'FixedObjectCd3': agate.text(),
        'FixedObjectCd4': agate.text(),
        'FixedObjectCd5': agate.text(),
        'FixedObjectCd6': agate.text(),
        'FixedObjectCd7': agate.text(),
        'FixedObjectCd8': agate.text()]
})

vehicles2015 = agate.Table.from_csv('vehicles2015.csv', column_types=tester)

SyntaxError: invalid syntax (<ipython-input-43-a8e5db22b42c>, line 3)

In [44]:
vehicles2015 = agate.Table.from_csv('vehicles2015.csv', 'FixedObjectCd1': agate.text())

SyntaxError: invalid syntax (<ipython-input-44-53c9dfac9617>, line 1)

In [73]:
## try pivot joined_all
transformed=joined_all.pivot('CrashType')
transformed.print_table()

|------------+--------|
|  CrashType | Count  |
|------------+--------|
|          3 |   336  |
|          9 |   283  |
|          7 | 7,309  |
|          5 |    47  |
|         10 |    20  |
|          1 |     9  |
|          2 |   107  |
|          4 |     6  |
|         11 |     6  |
|         16 |     4  |
|         17 |     1  |
|         13 |     8  |
|         18 |     2  |
|         15 |     1  |
|------------+--------|


In [74]:
## try pivot Distinct_CrashId
transformed_distinct = Distinct_CrashId('CrashType')
transformed_distinct.print_table()


TypeError: 'Table' object is not callable

In [ ]:
## create a new table with only two columns: CrashId, Object
fixed_objectcd1 = joined_all.select(('CrashId', 'FixedObjectCd1'))
object1=fixed_objectcd1.rename(column_names=['CrashId','FixedObject'])




objects = agate.TableSet([
        object1,
        object2,
        object3,
    ],
        ['cd1', 'cd2', 'cd3'],
    key_name='cd_num'
                        )

FixedObject = objects.merge(groups=None, group_name=None, group_type=None)
FixedObject_transformed = objects.pivot('FixedObject')
FixedObject_transformed.print_table()